# Домашняя работа к уроку 6. Задача look-alike

#### Домашнее задание

1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)

2. сделать feature engineering

3. обучить любой классификатор (какой вам нравится)

4. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть

5. применить random negative sampling для построения классификатора в новых условиях

6. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)

7. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

**Бонусный вопрос:**
Как вы думаете, какой из методов на практике является более предпочтительным: random negative sampling или 2-step approach?

Набор данных для бинарной классификации - банкрот или нет компания

Описание данных - https://archive.ics.uci.edu/ml/datasets/Qualitative_Bankruptcy#

Attribute Information: (P=Positive,A-Average,N-negative,B-Bankruptcy,NB-Non-Bankruptcy)

- Industrial Risk: {P,A,N}
- Management Risk: {P,A,N}
- Financial Flexibility: {P,A,N}
- Credibility: {P,A,N}
- Competitiveness: {P,A,N}
- Operating Risk: {P,A,N}
- Class: {B,NB}

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import pandas as pd
import numpy as np
data = pd.read_csv("/content/drive/MyDrive/Qualitative_Bankruptcy.data.txt", header=None)
data.columns=['Industrial Risk', 'Management Risk', 'Financial Flexibility', 
              'Credibility','Competitiveness', 'Operating Risk','Class']
data.head(3)

,Industrial Risk,Management Risk,Financial Flexibility,Credibility,Competitiveness,Operating Risk,Class
0,P,P,A,A,A,P,NB
1,N,N,A,A,A,N,NB
2,A,A,A,A,A,A,NB


Имеется 6 признаков и 1 целевая переменная - нужно определить является ли компания банкротом

In [15]:
print(data.shape)

(250, 7)


#### Сдедать feature engineering

In [16]:
#P=Positive,A-Average,N-negative
mapping = {'P': 1,'A': 0,'N': -1,'B': -1,'NB': 1}

df=pd.DataFrame()

#  = data.applymap(mapping)
for column in data.columns:
    df[column]=data[column].map(mapping)
    
# суммарный риск и 2 сумма (Credibility + Competitiveness)
df['F1'] = (df['Industrial Risk']+df['Management Risk']+df['Operating Risk'])
df['F2'] = (df['Credibility']+df['Competitiveness'])

# целевой столбец последний
cols = list(df.columns)
a, b = cols.index('Class'), cols.index('F2')
cols[b], cols[a] = cols[a], cols[b]
df = df[cols]

df.head(3)

,Industrial Risk,Management Risk,Financial Flexibility,Credibility,Competitiveness,Operating Risk,F2,F1,Class
0,1,1,0,0,0,1,0,3,1
1,-1,-1,0,0,0,-1,0,-3,1
2,0,0,0,0,0,0,0,0,1


In [17]:
df['Class'].value_counts()
# 1 = Не банкрот
# -1 = банкрот

 1    143
-1    107
Name: Class, dtype: int64

#### Обучить любой классификатор - KNeighborsClassifier

In [18]:
from sklearn.model_selection import train_test_split

y_data = df['Class']
x_data = df.drop(columns='Class')


x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [19]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=2)

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

Проверка качества

In [20]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 
    return [f1,roc,rec,prc]

result= evaluate_results(y_test, y_predict)
result.append('step4')
A = pd.DataFrame([result],columns=['f1', 'roc', 'recall', 'precision','Model'])

Classification results:
f1: 100.00%
roc: 100.00%
recall: 100.00%
precision: 100.00%


#### Разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть

In [21]:
share = .05

mod_data = df.copy()
#get the indices of the positives samples
P = np.where(mod_data['Class'].values == 1)[0]
#shuffle them
np.random.shuffle(P)
# leave just share% of the positives marked
P_len = int(np.ceil(share * len(P)))
print(f'Using {P_len}/{len(P)} as positives and unlabeling the rest')
P_sample = P[:P_len]

Using 8/143 as positives and unlabeling the rest


In [22]:
# столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)
mod_data['class_test'] = -1
mod_data.loc[P_sample,'class_test'] = 1
print('target variable:\n', mod_data['class_test'].value_counts())

target variable:
 -1    242
 1      8
Name: class_test, dtype: int64


8 значений отмечены как положительные, а остальные - отрицательные.

In [23]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.class_test.values # new class (just the P & U)
y_positive = mod_data.Class.values # original class

#### Применить random negative sampling для построения классификатора в новых условиях

In [24]:
mod_data = mod_data.sample(frac=1)

neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(8, 10) (8, 10)


In [25]:
model = KNeighborsClassifier(n_neighbors=2)

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)

result= evaluate_results(sample_test.iloc[:,-2].values, y_predict)
result.append('last step')
B = pd.DataFrame([result],columns=['f1', 'roc', 'recall', 'precision','Model'])

Classification results:
f1: 97.01%
roc: 96.15%
recall: 100.00%
precision: 94.20%


#### Сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)

In [26]:
AB = A.append(B)
AB = AB[['Model','f1', 'roc', 'recall', 'precision']]
AB

,Model,f1,roc,recall,precision
0,step4,1.000000,1.000000,1.0,1.000000
0,last step,0.970149,0.961538,1.0,0.942029


Качество решения из пункта 4 выше из-за наибольшей выборки.

#### Поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

Разница становится заметной при низких значениях выборки (от 5%) и скорее всего обусловлено сильной связью между факторами и предсказанием.